# LinkedIn Tech Jobs Dataset Analytics

*# This is actually a weekly assignment for my Big Data & Predictive Analytics, but I got pretty interested so I put quite the effort*

In [2]:
# Install PySpark
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285387 sha256=1944a08d7080add325169a380c49f62ced8b92a31bd437e734af6a55f8e4c2a6
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [3]:
# Mount Google Drive
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Import SparkSession from pyspark.sql
from pyspark.sql import SparkSession

# Create instance of SparkSession builder
spark = SparkSession.builder.appName('data_manipulation').getOrCreate()

In [5]:
# Validate spark session status

if (spark.getActiveSession()):
    print('yes')
else:
    print('no')

yes


In [6]:
# The DataFrame is from https://www.kaggle.com/datasets/joebeachcapital/linkedin-jobs

# Load dataset as PySpark DataFrame
linkinjobs = spark.read.csv("/content/drive/linkedin_job.csv", header = True, inferSchema = True)

# Show 20 top rows
linkinjobs.show()

+------------+-------+--------------------+------------+----------------+------------------+----------+-----------------+--------------+--------------------+------+---+----+------+-----+-----+------+-----+-----+---+---+-----+---+-------+----+----------+-----+----+----------+------+-----+-------+---+---+-------+------+-----+--------+--------+----+---+
|Company_Name|  Class|         Designation|    Location|Total_applicants|LinkedIn_Followers|     Level|      Involvement|Employee_count|            Industry|PYTHON|C++|JAVA|HADOOP|SCALA|FLASK|PANDAS|SPARK|NUMPY|PHP|SQL|MYSQL|CSS|MONGODB|NLTK|TENSORFLOW|LINUX|RUBY|JAVASCRIPT|DJANGO|REACT|REACTJS| AI| UI|TABLEAU|NODEJS|EXCEL|POWER BI|SELENIUM|HTML| ML|
+------------+-------+--------------------+------------+----------------+------------------+----------+-----------------+--------------+--------------------+------+---+----+------+-----+-----+------+-----+-----+---+---+-----+---+-------+----+----------+-----+----+----------+------+-----+------

In [7]:
# Print columns' names and datatypes
linkinjobs.printSchema()


root
 |-- Company_Name: string (nullable = true)
 |-- Class: string (nullable = true)
 |-- Designation: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Total_applicants: integer (nullable = true)
 |-- LinkedIn_Followers: integer (nullable = true)
 |-- Level: string (nullable = true)
 |-- Involvement: string (nullable = true)
 |-- Employee_count: integer (nullable = true)
 |-- Industry: string (nullable = true)
 |-- PYTHON: integer (nullable = true)
 |-- C++: integer (nullable = true)
 |-- JAVA: integer (nullable = true)
 |-- HADOOP: integer (nullable = true)
 |-- SCALA: integer (nullable = true)
 |-- FLASK: integer (nullable = true)
 |-- PANDAS: integer (nullable = true)
 |-- SPARK: integer (nullable = true)
 |-- NUMPY: integer (nullable = true)
 |-- PHP: integer (nullable = true)
 |-- SQL: integer (nullable = true)
 |-- MYSQL: integer (nullable = true)
 |-- CSS: integer (nullable = true)
 |-- MONGODB: integer (nullable = true)
 |-- NLTK: integer (nullable = true)

In [8]:
# Swap column names of Level and Involvement as they seem to be mismatched with the column values
linkinjob_rev = linkinjobs.withColumnRenamed("Involvement", "LevelTemp").withColumnRenamed("Level", "Involvement").withColumnRenamed("LevelTemp", "Level")\
.withColumnRenamed("Total_applicants", "Total_Applicants").withColumnRenamed("Employee_count", "Employee_Count")

# Print swapped dataframe top rows
linkinjob_rev.show()

+------------+-------+--------------------+------------+----------------+------------------+-----------+-----------------+--------------+--------------------+------+---+----+------+-----+-----+------+-----+-----+---+---+-----+---+-------+----+----------+-----+----+----------+------+-----+-------+---+---+-------+------+-----+--------+--------+----+---+
|Company_Name|  Class|         Designation|    Location|Total_Applicants|LinkedIn_Followers|Involvement|            Level|Employee_Count|            Industry|PYTHON|C++|JAVA|HADOOP|SCALA|FLASK|PANDAS|SPARK|NUMPY|PHP|SQL|MYSQL|CSS|MONGODB|NLTK|TENSORFLOW|LINUX|RUBY|JAVASCRIPT|DJANGO|REACT|REACTJS| AI| UI|TABLEAU|NODEJS|EXCEL|POWER BI|SELENIUM|HTML| ML|
+------------+-------+--------------------+------------+----------------+------------------+-----------+-----------------+--------------+--------------------+------+---+----+------+-----+-----+------+-----+-----+---+---+-----+---+-------+----+----------+-----+----+----------+------+-----+---

In [9]:
# Print dataframe rows and columns count
print((linkinjob_rev.count(),len(linkinjob_rev.columns)))

(811, 41)


# Selecting


In [10]:
# Selecting few columns of dataframe into a new dataframe
selected1 = linkinjob_rev.select('Company_Name', 'Designation', 'Total_Applicants', 'Location', 'Level')

# Print dataframe rows sorted by Total_Applicants descending
print("Jobs by Total Applicants")
selected1.orderBy(['Total_Applicants'], ascending=False).show()

Jobs by Total Applicants
+--------------------+--------------------+----------------+------------+-----------------+
|        Company_Name|         Designation|Total_Applicants|    Location|            Level|
+--------------------+--------------------+----------------+------------+-----------------+
|Tata Consultancy ...|             Node JS|             196|   Karnataka| Mid-Senior level|
|               Wipro|           Developer|             191| Maharashtra|      Entry level|
|             Infosys|Technology Analys...|             191|     Haryana|        Associate|
|               Wipro|    Business Analyst|             186| Maharashtra| Mid-Senior level|
|IDESLABS PRIVATE ...|       Manual Tester|             178|   Telangana|      Entry level|
|             Infosys|    Dotnet Developer|             170|      Kerala| Mid-Senior level|
|Tata Consultancy ...|              Python|             168|   Telangana|        Associate|
|IDESLABS PRIVATE ...|    Delivery Manager|            

In [12]:
# Import col from PySpark sql functions
from pyspark.sql.functions import col

# Selecting few columns of dataframe
selected2 = linkinjob_rev.select('Company_Name', 'Designation', 'Total_Applicants', 'JAVA')

# Change column name JAVA to Java
selected2_1 = selected2.withColumnRenamed("JAVA", "Java")

# Print dataframe rows with jobs that require Java Skill
print("Java Jobs")
selected2_1.where(col("Java") == 1).show()

Java Jobs
+------------+--------------------+----------------+----+
|Company_Name|         Designation|Total_Applicants|Java|
+------------+--------------------+----------------+----+
|       Wipro|     Project Manager|              52|   1|
|       Wipro|           Developer|             156|   1|
|       Wipro|       Test Engineer|               0|   1|
| LTIMindtree|      Java Developer|              22|   1|
| LTIMindtree|      Java Developer|               6|   1|
| LTIMindtree|PLSQL Development...|              48|   1|
| LTIMindtree|      Java Developer|               0|   1|
| LTIMindtree|      Java Developer|               6|   1|
| LTIMindtree|      Java Developer|               5|   1|
| LTIMindtree|      APEX Developer|              12|   1|
| LTIMindtree|      Java Developer|               2|   1|
| LTIMindtree|Java Back-end Dev...|              25|   1|
| LTIMindtree|Infra Cloud Archi...|               0|   1|
| LTIMindtree|Looking for PHP D...|              36|   1|
| LT

# Filtering

In [13]:
# When I use df.column_name and "==" operand my code runs but the rows are blank,
# I also tried df['column_name'] to address the column but I got same result, df.select("column_name") doesn't work either
# then I found df.column_name.contains("string") that got the job done


# Filter for Level that contains "Entry level"
filterA = linkinjob_rev.Level.contains("Entry level")

# Filter for jobs that have more than 1 applicants
filterB = linkinjob_rev.Total_Applicants > 1

# Applying the filter to a new dataframe
selected2 = selected1.filter(filterA).filter(filterB).orderBy(['Total_Applicants'], ascending=True)

# Print dataframe top rows
selected2.show()

+--------------------+--------------------+----------------+--------------+------------+
|        Company_Name|         Designation|Total_Applicants|      Location|       Level|
+--------------------+--------------------+----------------+--------------+------------+
|IDESLABS PRIVATE ...|          AWS DevOps|               2|     Telangana| Entry level|
| ACURA SOLUTIONS LTD|Hiring for Associ...|               2| Uttar Pradesh| Entry level|
|IDESLABS PRIVATE ...|Associate Tech Sp...|               2|   Maharashtra| Entry level|
|         LTIMindtree|      Java Developer|               2|    Tamil Nadu| Entry level|
|IDESLABS PRIVATE ...|Associate Tech Sp...|               2|   Maharashtra| Entry level|
|         LTIMindtree| SAP Data Specialist|               2|     Karnataka| Entry level|
|IDESLABS PRIVATE ...| .Net Core Developer|               2|   Maharashtra| Entry level|
|         LTIMindtree|Java Full Stack w...|               2|   Maharashtra| Entry level|
|IDESLABS PRIVATE ...

In [20]:
# Import method StringType to convert column datatype to String
from pyspark.sql.types import StringType

# Selecting some columns and renaming JAVA to Java_Skill
selected3 = linkinjob_rev.select("Company_Name", "Location", "Total_Applicants", "Level", "JAVA").withColumnRenamed("JAVA", "Java_Skill")

# Convert Java_Skill column to String
selected33 = selected3.withColumn("Java_Skill", selected3["Java_Skill"].cast(StringType()))

# Replace '1' value with 'Yes' in Java_Skill
selected3new = selected33.replace(['1'],['Yes'],'Java_Skill')

# Replace '0' value with 'No' in Java_Skill
selected3new1 = selected3new.replace(['0'],['No'],'Java_Skill')

# Filter for Location == "Delhi"
filterC = selected3new1.Location.contains("Delhi")

# Filter for Java_Skill == "Yes"
filterD = selected3new1.Java_Skill.contains("Yes")

# Applying the filter to a new dataframe
selected4 = selected3new1.filter(filterC).filter(filterD)

# Print dataframe top rows
selected4.show()

+--------------------+--------+----------------+-----------------+----------+
|        Company_Name|Location|Total_Applicants|            Level|Java_Skill|
+--------------------+--------+----------------+-----------------+----------+
|         LTIMindtree|   Delhi|               0|      Entry level|       Yes|
|         LTIMindtree|   Delhi|               0| Mid-Senior level|       Yes|
|         LTIMindtree|   Delhi|              36|      Entry level|       Yes|
|         LTIMindtree|   Delhi|               2|         Director|       Yes|
|         LTIMindtree|   Delhi|               2|      Entry level|       Yes|
|         LTIMindtree|   Delhi|               3|      Entry level|       Yes|
|         LTIMindtree|   Delhi|               7|      Entry level|       Yes|
|         LTIMindtree|   Delhi|              19|      Entry level|       Yes|
|IDESLABS PRIVATE ...|   Delhi|              11|      Entry level|       Yes|
| ACURA SOLUTIONS LTD|   Delhi|              18|      Entry leve

# Aggregating

In [21]:
# Print describe method for Total_Applicants, LinkedIn_Followers and Employee_Count
linkinjob_rev.describe('Total_Applicants', "LinkedIn_Followers", "Employee_Count").show()

# Select the max value for Total_Applicants and save the value
max_applicants = linkinjob_rev.agg({"Total_Applicants": "max"}).collect()[0][0]

# Filter row where max Total_Applicant is
max_applicants_row = linkinjob_rev.filter(col("Total_Applicants") == max_applicants)

# Print the row
max_applicants_row.show()

+-------+------------------+------------------+-----------------+
|summary|  Total_Applicants|LinkedIn_Followers|   Employee_Count|
+-------+------------------+------------------+-----------------+
|  count|               811|               811|              811|
|   mean|23.432799013563503|1401891.1134401972|5177.552404438964|
| stddev| 35.13777271035743|2688592.5313296667| 4194.16958135799|
|    min|                 0|             11245|              110|
|    max|               196|          11996967|            10001|
+-------+------------------+------------------+-----------------+

+--------------------+-------+-----------+----------+----------------+------------------+-----------+-----------------+--------------+--------------------+------+---+----+------+-----+-----+------+-----+-----+---+---+-----+---+-------+----+----------+-----+----+----------+------+-----+-------+---+---+-------+------+-----+--------+--------+----+---+
|        Company_Name|  Class|Designation|  Location|T

# Looping Through Row and Columns to Append Needed Skill to An Array then Returning the Array as a PySpark Dataframe Column

Originally, after looking at this dataset, I immediately thought about adding a column for the qualifications/skills required for a job description in the dataset. I tried to write the code for it, but it turned out to be quite complex and challenging. So, in the end, I decided to seek AI assistance in creating the code. The code should function by looping through specific rows and columns where programming skills/languages are indicated (with the language column having values of 0 and 1). If a value of 1 (true) is found, it should append the name of the language column with a value of true to an array called "qualification." Once the looping is complete, the function should return this array as a new column in the dataframe. This way, we can identify the programming languages or qualifications required for the job description in that particular row. By creating this new column, we can explore the dataset further and obtain more comprehensive and clear information.

In [23]:
# Import needed library

import numpy as np
from pyspark.sql.functions import col, create_map
from pyspark.sql.types import MapType, ArrayType, StringType

In [24]:
# Define specific columns to check for the value 1
columns_to_check = linkinjob_rev.columns[10:]

# Function to find qualifications for a row
def find_qualifications(*cols):
    qualifications = []
    for i, col_name in enumerate(columns_to_check):
        if cols[i] == 1:
            qualifications.append(col_name)
    return qualifications

# Define a UDF (User Defined Function) to apply the find_qualifications function
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType

find_qualifications_udf = udf(find_qualifications, ArrayType(StringType()))

# Add the "qualification" column to the DataFrame
linkinjob_qualification = linkinjob_rev.withColumn("Qualification", find_qualifications_udf(*[col(col_name) for col_name in columns_to_check]))

# Show the DataFrame with the added "qualification" column
linkinjob_qualification.show(truncate=False)

+------------+-------+---------------------+------------+----------------+------------------+-----------+-----------------+--------------+------------------------------+------+---+----+------+-----+-----+------+-----+-----+---+---+-----+---+-------+----+----------+-----+----+----------+------+-----+-------+---+---+-------+------+-----+--------+--------+----+---+--------------------------+
|Company_Name|Class  |Designation          |Location    |Total_Applicants|LinkedIn_Followers|Involvement|Level            |Employee_Count|Industry                      |PYTHON|C++|JAVA|HADOOP|SCALA|FLASK|PANDAS|SPARK|NUMPY|PHP|SQL|MYSQL|CSS|MONGODB|NLTK|TENSORFLOW|LINUX|RUBY|JAVASCRIPT|DJANGO|REACT|REACTJS|AI |UI |TABLEAU|NODEJS|EXCEL|POWER BI|SELENIUM|HTML|ML |Qualification             |
+------------+-------+---------------------+------------+----------------+------------------+-----------+-----------------+--------------+------------------------------+------+---+----+------+-----+-----+------+-----

# Exploring Further

In [25]:
# Import the size function for counting the number of elements in an array in PySpark
from pyspark.sql.functions import size

# Creating a new dataframe with the result of counting the number of programming language elements required for a job description
languages = linkinjob_qualification.withColumn("Languages_Count", size("Qualification"))

languages_sel = languages.select("Company_Name", "Designation", "Location", "Total_Applicants", "Qualification", "Languages_Count")

sorted_languages = languages_sel.orderBy("Total_Applicants", ascending=False)

sorted_languages.show(truncate=False)

+-------------------------+----------------------------------------+------------+----------------+---------------------------------------------------------+---------------+
|Company_Name             |Designation                             |Location    |Total_Applicants|Qualification                                            |Languages_Count|
+-------------------------+----------------------------------------+------------+----------------+---------------------------------------------------------+---------------+
|Tata Consultancy Services|Node JS                                 | Karnataka  |196             |[JAVA, JAVASCRIPT, AI, UI]                               |4              |
|Wipro                    |Developer                               | Maharashtra|191             |[AI, UI, EXCEL]                                          |3              |
|Infosys                  |Technology Analyst - .Net               | Haryana    |191             |[AI, UI]                             

In [26]:
sorted_languages2 = languages_sel.orderBy("Languages_Count", ascending=False)

sorted_languages2.show(truncate=False)

+-------------------+--------------------------------------------------------------------+--------------+----------------+-----------------------------------------------------------------------------------------+---------------+
|Company_Name       |Designation                                                         |Location      |Total_Applicants|Qualification                                                                            |Languages_Count|
+-------------------+--------------------------------------------------------------------+--------------+----------------+-----------------------------------------------------------------------------------------+---------------+
|Uplers             |Python Software Engineer                                            | Tamil Nadu   |35              |[PYTHON, JAVA, FLASK, SQL, MYSQL, CSS, MONGODB, JAVASCRIPT, DJANGO, AI, UI, HTML, ML]    |13             |
|Uplers             |Python Software Engineer                                       

In [35]:
# We can even drop the programming language columns as we have already saved it in the Qualification column

# Creating indexes for columns from Industry to ML
columns_to_drop_indices = list(range(9, 41))

# Copying the dataframe that already contains the Languages_Count column
lang_todrop = languages

# Retrieve the column names from index 9 to 41
column_names_to_drop = [lang_todrop.columns[i] for i in columns_to_drop_indices]

# Drop columns with the collected names
lang_dropped = lang_todrop.drop(*column_names_to_drop)

# Print dataframe with dropped columns
lang_dropped.show(truncate=False)

+------------+-------+---------------------+------------+----------------+------------------+-----------+-----------------+--------------+--------------------------+---------------+
|Company_Name|Class  |Designation          |Location    |Total_Applicants|LinkedIn_Followers|Involvement|Level            |Employee_Count|Qualification             |Languages_Count|
+------------+-------+---------------------+------------+----------------+------------------+-----------+-----------------+--------------+--------------------------+---------------+
|Wipro       |Class 3|Project Manager      | Maharashtra|0               |6737300           |Full-time  | Entry level     |10001         |[SCALA, AI, UI, EXCEL, ML]|5              |
|Wipro       |Class 3|Project Manager      | Delhi      |121             |6737300           |Full-time  | Entry level     |10001         |[SCALA, AI, UI, EXCEL, ML]|5              |
|Wipro       |Class 3|Project Manager      | Delhi      |0               |6737300         

In [38]:
# Import array_contains function to check array element
from pyspark.sql.functions import array_contains

# Filter for Qualification column that contains JAVA
by_lang = lang_dropped.filter(array_contains(lang_dropped.Qualification, "JAVA"))

# Print filtered dataframe
by_lang.show(truncate=False)

+------------+-------+-------------------------------------------+------------+----------------+------------------+-----------+-----------------+--------------+---------------------------------------------------------+---------------+
|Company_Name|Class  |Designation                                |Location    |Total_Applicants|LinkedIn_Followers|Involvement|Level            |Employee_Count|Qualification                                            |Languages_Count|
+------------+-------+-------------------------------------------+------------+----------------+------------------+-----------+-----------------+--------------+---------------------------------------------------------+---------------+
|Wipro       |Class 3|Project Manager                            | West Bengal|52              |6737300           |Full-time  | Entry level     |10001         |[JAVA, AI, UI, SELENIUM]                                 |4              |
|Wipro       |Class 3|Developer                             